# Model Inference

## Installation

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!pip install transformers
!pip install pysrt
!pip install spacy
# !pip install torch
# !pip3 install tensorflow torch
# !pip install tensorflow
# !pip install pytorch


     |████████████████████████████████| 5.3 MB 742 kB/s eta 0:00:01
     |████████████████████████████████| 163 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 744 kB/s eta 0:00:01
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13444 sha256=c1f33b164dc7fa613728b18561106bc5c39b6f79da9473c791139ffb0be9b8d8
  Stored in directory: /Users/manideepchakilam/Library/Caches/pip/wheels/c3/34/f1/ae1d86b7f454100c10f7ab8dc411303b7834e7f40e343ca2c0
Successfully built pysrt
     |████████████████████████████████| 6.7 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 90.4 MB/s eta 0:00:01
     |████████████████████████████████| 491 kB 99.9 MB/s eta 0:00:01
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached pathy-0.6.2-py3-none-any.whl (42 kB)
  Using cached typer-0.4.2-py3-none-

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/MyDrive')
os.getcwd()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from configs.modelConfig import modeln
import pandas as pd

In [9]:
import pandas as pd
df_prof = pd.read_excel(r'Data/prof_blasph_data.xlsx', sheet_name= 'Profanity', engine='openpyxl')
df_prof.head(10)

Profanity                                           Sentence
0         Ass                                I sweat my ass off.
1     Asshole                          You really are an asshole
2      Ballsy  Whoever could pull off such a ballsy move as t...
3     Bastard             That bastard! Claire wanted to scream.
4       Bitch                      I know them bitch ass niggas.
5       Boobs  The young policeman still couldn't keep his ey...
6    Bullshit                        That myth is pure bullshit.
7  Cocksucker  After the FBI finished with me, I was handed o...
8        Crap                             I'm sick of this crap.
9      Crappy                     You see, his diet was crappy!.

In [10]:
df_blasph = pd.read_excel(r'Data/prof_blasph_data.xlsx', sheet_name= 'Blasphemy', engine='openpyxl')
df_blasph.head(10)

Blasphemy                                           Sentence
0             Christ                             Christ, I was an idiot
1  For Christ’s Sake  Would you let me finish my story, for Christ's...
2     For God’s Sake       Someone broke into her place for god's sake!
3   For Satan’s Sake                  And wear it long for Satan's sake
4                God                           Oh God! That is so good.
5           God Damn                 Put the God Damn phone in the bag.
6          Goddamnit            No, Goddamnit, he didn't kill some FAC.
7          Good Lord  I find his grace my very good lord indeed; and...
8         Hail Satan           All Hail Satan, my Dark Lord and Master.
9               Hell                               To hell with it all.

## creating  dataset

In [11]:
text = (' '.join(df_prof.Sentence)) #+ ' ' + ' '.join(df_blas.Sentence))
# text = (' '.join(df_blas.Sentence))

doc = nlp(text)
words = []
labels = []

for token in doc:
    words.append(token.text)
    labels.append('O')

df = pd.DataFrame({'word': words, 'label': labels})
df.to_csv('token_per_line.biluo', index=False)

In [12]:
for j in range(0, len(df2)):
    list_df = list(map(lambda x: x.lower(), list(df_prof.Profanity)))
    if df2.loc[j,'word'].lower() in list_df:
        df2.loc[j,'label'] = 'U-PROF'

In [15]:
dpath = 'Data/token_per_line_profanity.biluo'

df2 = pd.read_csv(dpath, sep=',')
df2.head()

word   label
0      I       O
1  sweat       O
2     my       O
3    ass  U-PROF
4    off       O

In [14]:
dpath = 'Data/token_per_line_blasphemy.biluo'

df1 = pd.read_csv(dpath, sep=',')
df1.head()

word    label
0  Christ  U-BLASP
1       I        O
2     was        O
3      an        O
4   idiot        O

In [16]:
df2 = df2.append(df1).reset_index(drop = True)

# Later Prepocessing

In [ ]:
words  = df2.word.values

ents = df2.label.values
text = ' '.join(words)
text =  text.replace("’","")
text =  text.replace("'","")


In [ ]:
from spacy.gold import GoldParse

doc = nlp.make_doc(text)
doc

# Gold Parsing Before Training

In [ ]:
g = GoldParse(doc, entities=ents)
X = [doc]
Y = [g]

# Final Pipelines

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(f'[OtherPipes] = {other_pipes} will be disabled')

In [ ]:
add_ents = ['PROF','BLASP'] # The new entity
# Piplines in core pretrained model are tagger, parser, ner. Create new if blank model is to be trained using `spacy.blank('en')` else get the existing one.
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner") # "architecture": "ensemble" simple_cnn ensemble, bow # https://spacy.io/api/annotation
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")
prev_ents = ner.move_names # All the existing entities recognised by the model
print('[Existing Entities] = ', ner.move_names)
for ent in add_ents:
    ner.add_label(ent)


In [ ]:
new_ents = ner.move_names
# print('\n[All Entities] = ', ner.move_names)
print('\n\n[New Entities] = ', list(set(new_ents) - set(prev_ents)))
## Training
model = None # Since we are training a fresh model not a saved model


In [ ]:
n_iter = 50
with nlp.disable_pipes(*other_pipes):  # only train ner
    # optimizer = nlp.begin_training()
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    for i in range(n_iter):
        losses = {}
        nlp.update(X, Y,  sgd=optimizer, drop=0.0, losses=losses)
        # nlp.entity.update(d, g)
        print("Losses", losses)
        

In [ ]:
# Output directory
from pathlib import Path
output_dir=Path( r'/model/')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'prof'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Main Inference

In [17]:
import pysrt
def sent_Classifier(sent, model):
    words = []
    pred = []
    result = model(sequences = sent, 
                                  candidate_labels = ["Profanity", "Blasphemy"], 
                                  multi_class= True)

    return result


In [24]:
sent_Classifier("Fucking dance is going on", modeln)

{'labels': ['Profanity', 'Blasphemy'],
 'scores': [0.9956592321395874, 0.0009783301502466202],
 'sequence': 'Fucking dance is going on'}

In [19]:
def main(srt_file_path, srt_file_name):
    df1 = pd.DataFrame(columns = ["text", "start", "end"])
    df2 = pd.DataFrame(columns= [ "Profanity", "Blasphemy"])
    subs = pysrt.open(srt_file_path + srt_file_name)
    for sub in subs:
        result = sent_Classifier(sub.text,modeln)
        print(sub.text)
        # df = df.append({'text': sub.text, 'start':sub.start, 'end': sub.end, 'class': ','.join(result["labels"]), "scores": result["scores"]}, ignore_index=True )
        s1 = pd.Series([sub.text, sub.start, sub.end], index=df1.columns)
        df1 = df1.append(s1, ignore_index=True)

        s2 = pd.Series(result["scores"], index=result["labels"])
        df2 = df2.append(s2, ignore_index=True)
    result = pd.concat([df1, df2], axis=1)
    return result

In [20]:
srt_file_path = r"/content/drive/MyDrive/BigDataProject/srt_files/"
srt_file_name = r"Bully 720pED.srt"

data_output = main(srt_file_path, srt_file_name)

I want you
to suck my big dick.
Marty honey, dinner.
I want you
to lick my balls.
<i>His cock?</i>
<i>His cock was beautiful.</i>
And he ate me out
for an hour.
Lisa, you know,
it's all about attitude.
If you want it, you can get it.
You just got to get
your fuckin' ass on the beach.
- Work for it.
- Whatever.
Excuse me,
sandwich boy?
Aren't you supposed
to wear gloves,
'cause I don't know
where those hands have been.
Fuckin' pigs, man!
- My name's Marty, what's yours?
- Just give me the sub, asshole.
Ask the bitches to meet us,
fuckhole! We'll party.
We're going surfing, at 3:00.
If you want to meet us
at North Beach.
Maybe we'll think about it.
Great, babe,
you can be my date.
My friend here
will take that one.
Hey, fuck you, man.
- Those guys were fine.
- He was making fun of me.
No shit. He was just being
an asshole. Guys are like that.
But the other one,
the shorter one--
- The cute one.
- Yeah, Marty.
He acted like
he wanted to meet you.
- Meet you, you mean.
- No, his buddy's go

In [21]:
data_output

text         start  ... Profanity  Blasphemy
0         I want you\nto suck my big dick.  00:00:55,797  ...  0.952869   0.017106
1                     Marty honey, dinner.  00:00:59,334  ...  0.085922   0.000373
2            I want you\nto lick my balls.  00:01:08,443  ...  0.980064   0.066695
3                         <i>His cock?</i>  00:01:41,643  ...  0.761221   0.022771
4           <i>His cock was beautiful.</i>  00:01:47,849  ...  0.018840   0.000072
..                                     ...           ...  ...       ...        ...
95                          Fuck you, man.  00:11:18,920  ...  0.997905   0.211231
96         I'm sorry, man.\nLet's just go.  00:11:31,566  ...  0.555482   0.169429
97                    Let's go, I'm sorry.  00:11:35,536  ...  0.656767   0.108743
98  Get your fuckin' hands\noff me, Bobby.  00:11:37,438  ...  0.996864   0.654028
99                       I'm really sorry.  00:11:40,641  ...  0.282016   0.118824

[100 rows x 5 columns]